In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.geeksforgeeks.org/c-plus-plus/?ref=shm"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')


In [3]:
urls = []
pages = soup.find('ul', {'class': 'leftBarList'}).findAll(href=True)
for slug in pages[1:]:
    urls.append(slug.get('href'))

mainHeadings = []
subHeadingsH2 = []
subHeadingsH3 = []
imageURLs = []
videoURLs = []
links = []
facts = []
definitions = []
explanations = []
codes = []
outputs = []
listItems = []
tableData = []
data = {}


In [4]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html5lib')

    subContent = soup.find('article', {"class": "content"})  # Page Sub Content
    if subContent is None:
        continue

    mainHeading = subContent.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images[:-1]:
            pageImageURLs.append(img.get('src'))

    videos = subContent.findAll('iframe')  # Videos
    pageVideoURLs = []
    if videos is not None:
        for vid in videos:
            pageVideoURLs.append(vid.get('src'))

    link = subContent.findAll('a')  # Page Links
    pageLinks = []
    if link is not None:
        for lin in link:
            if lin.get('href') is not None and not lin.get('href').startswith('http'):
                pageLinks.append('https://www.geeksforgeeks.org/' + lin.get('href'))
            else:
                pageLinks.append(lin.get('href'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text.replace('\n', '')
            if 10 <= len(slug) <= 99:
                pageFacts.append(slug)
            elif 100 <= len(slug) <= 399:
                pageDefinitions.append(slug)
            elif 400 <= len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('div', {"class": "container"})  # Codes
    pageCodes = []
    if codeContent is not None:
        for slug in codeContent:
            pageCodes.append(slug.text.replace('\xa0', ' ').lstrip().rstrip())

    outputContent = subContent.findAll('pre')  # Outputs
    pageOutputs = []
    if outputContent is not None:
        for slug in outputContent:
            if slug.text != '':
                pageOutputs.append(slug.text.lstrip().rstrip())

    tab = subContent.findAll('figure', {"class": "table"})  # Tables
    tables = []
    for t in tab:
        tables.append(t.table)
    pageTableData = []
    if tables is not None:
        for tb in tables:
            final = ''
            for td in tb.findAll('tr'):
                table = ''
                row = ''
                data = td.findAll('td')
                if len(data) != 0:
                    for d in data:
                        row = row + str(d.text) + '|||'
                    table = table + row[:-3:1] + '&&&'
                    final = final + table
            if final != '':
                pageTableData.append(final[:-3:1])

    allListUL = subContent.findAll('ul')[1:]  # List Items
    allListOL = subContent.findAll('ol')
    listItem = ""
    ulListItem = ""
    olListItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            ulListItem = ''
            for ulLis in allListUL:
                listUL = ""
                for lis in ulLis.findAll('li'):
                    listUL = listUL + str(lis.text).replace('\xa0', '') + '|||'
                ulListItem = ulListItem + listUL[:-3:1] + '&&&'
        if len(allListOL) != 0:
            olListItem = ''
            for olLis in allListOL:
                listOL = ""
                for lis in olLis.findAll('li'):
                    listOL = listOL + str(lis.text).replace('\xa0', '') + '|||'
                olListItem = olListItem + listOL[:-3:1] + '&&&'
        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + ulListItem[:-3:1]
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + olListItem[:-3:1]
        else:
            listItem = listItem + ulListItem + '&&&' + olListItem
        pageListItems.append(listItem)



    mainHeadings.append(pageMainHeadings)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    imageURLs.append(pageImageURLs)
    videoURLs.append(pageVideoURLs)
    links.append(pageLinks)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    codes.append(pageCodes)
    outputs.append(pageOutputs)
    tableData.append(pageTableData)
    listItems.append(pageListItems)

In [5]:
data = {'main_headings': mainHeadings, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'image_urls': imageURLs, 'video_urls': videoURLs, 'links': links, 'facts': facts, 'definitions': definitions,
        'explanations': explanations, 'codes': codes, 'outputs': outputs, 'table_data': tableData,
        'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('GeeksforGeeks_Cpp.csv')

In [6]:
a=pd.read_csv("GeeksforGeeks_Cpp.csv")
a

,Unnamed: 0,main_headings,sub_headings_h2,sub_headings_h3,image_urls,video_urls,links,facts,definitions,explanations,codes,outputs,table_data,list_items
0,0,['Introduction to C++ Programming Language'],[],[],['https://media.geeksforgeeks.org/wp-content/c...,[],"['https://www.geeksforgeeks.org/easy/', 'https...",['Some of the features & key-points to note ab...,['C++ is a general-purpose programming languag...,[],[],[],[],['Simple: It is a simple language in the sense...
1,1,['Setting up C++ Development Environment'],['CPP'],[],[],['https://www.youtube.com/embed/dsbj7XpHOcE?li...,"['https://www.geeksforgeeks.org/basic/', 'http...",['Setting up local environment'],['For setting up your own personal development...,['C++ is a general-purpose programming languag...,['#include<iostream>using namespace std;main()...,['sudo apt-get update\nsudo apt-get install gc...,[],['Linux Installation: We will install the GNU ...
2,2,['C++ Programming Basics'],[],[],[],[],"['https://www.geeksforgeeks.org/easy/', 'https...",['Refer here for the archive of Basics in C++....,['C++ is a general-purpose programming languag...,[],[],[],[],['Basic Syntax and First Program in C++: Learn...
3,3,['C++ Data Types'],['CPP'],[],['https://media.geeksforgeeks.org/wp-content/c...,[],"['https://www.geeksforgeeks.org/basic/', 'http...",['Data types in C++ is mainly divided into thr...,['All variables use data-type during declarati...,['This article is contributed by Harsh Agarwal...,['// C++ program to sizes of data types#includ...,['Size of char : 1 byte\nSize of int : 4 bytes...,[],['Integer|||Character|||Boolean|||Floating Poi...
4,4,['Variables in C++'],['CPP'],[],['https://media.geeksforgeeks.org/wp-content/u...,[],"['https://www.geeksforgeeks.org/easy/', 'https...",['A variable is a name given to a memory locat...,['A variable name can consist of alphabets (bo...,[],"[""#include <iostream>using namespace std; int...",['// Declaring a single variable\ntype variabl...,[],['The value stored in a variable can be change...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,['Unordered Sets in C++ Standard Template Libr...,"['CPP', 'CPP']",[],[],[],"['https://www.geeksforgeeks.org/easy/', 'http:...",['Below is C++ solution using unordered_set.\x...,['Recent articles on unordered_setThis article...,['An unordered_set is implemented using a hash...,['// C++ program to demonstrate various functi...,['slow not found\n\nFound c++\n\nAll elements ...,[],['insert()– Insert a new {element} in the unor...
66,66,['Multiset in C++ Standard Template Library (S...,"['CPP', 'C++']",['List of Functions of Multiset'],[],['https://www.youtube.com/embed/xelzlR_OGnI?fe...,"['https://www.geeksforgeeks.org/easy/', 'https...","['Implementation:\xa0', '\xa0Removing Element ...",['Multisets are a type of associative containe...,[],['// CPP Program to demonstrate the// implemen...,['The multiset gquiz1 is : \n60 50 50 40 30 20...,['begin()|||Returns an iterator to the first e...,['begin() – Returns an iterator to the first e...
67,67,['Map in C++ Standard Template Library (STL)'],['CPP'],['List of all Functions of Map\xa0'],[],['https://www.youtube.com/embed/kDwXAmLz47w?fe...,"['https://www.geeksforgeeks.org/medium/', 'htt...",['Some basic functions associated with Map:\xa...,['Maps are associative containers that store e...,[],['// CPP Program to demonstrate the implementa...,['The map gquiz1 is : \n KEY ELEMENT\n ...,['map::insert()|||Insert elements with a parti...,['begin() – Returns an iterator to the first e...
68,68,['Advanced C++ with Boost Library'],"['CPP', 'CPP', 'CPP']",[],[],[],"['https://www.geeksforgeeks.org/medium/', 'htt...","['Installation', 'Example Applications', 'Belo...",['Boost Libraries are intended to be widely us...,['This article is contributed by Shubham Bansa...,['// CPP Program to demonstrate Big Integer da...,['Product of 98745636214564698 * 7459874565236...,[],[]
